In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [3]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

NUM_TRAIN = 49000
NUM_VAL = 1000

cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=T.ToTensor())
loader_train = DataLoader(cifar10_train, batch_size=64, sampler=ChunkSampler(NUM_TRAIN, 0))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=T.ToTensor())
loader_val = DataLoader(cifar10_val, batch_size=64, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True,
                          transform=T.ToTensor())
loader_test = DataLoader(cifar10_test, batch_size=64)


Files already downloaded and verified


Files already downloaded and verified


Files already downloaded and verified


In [142]:
dtype = torch.cuda.FloatTensor # the CPU datatype

# Constant to control how frequently we print train loss
print_every = 100

# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

In [194]:
def train(model, loss_fn, optimizer, num_epochs = 1):
    best_val_acc = 0.
    
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        
        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.type(dtype))
            y_var = Variable(y.type(dtype).long())

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        val_acc = check_accuracy(net, loader_val)
        
        if val_acc > best_val_acc:
            torch.save(net, 'net_%d.model' % (val_acc*100))

def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_var = Variable(x.type(dtype), volatile=True)

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    
    return acc

In [213]:
class BaseNet(nn.Module):
    def __init__(self, use_dropout=False, p=0.9, input_shape=(3, 32, 32)):
        super(BaseNet, self).__init__()
        
        print(use_dropout, p, input_shape)
        assert len(input_shape) == 3, 'Input shape must be provided'
        
        self.input_channels = input_shape[0]
        self.output_classes = 10
        self.input_shape = input_shape
        num_layers = 10
        
        layers = []
        
        for i in range(num_layers):
            inp_channels = self.input_channels if i == 0 else 64
            
            layers += [
                nn.Conv2d(inp_channels, 64, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(64)
            ]
            
            if i == 2 or i == 5 or i == 8:
                layers.append(nn.MaxPool2d(2))
            
            if use_dropout:
                layers.append(nn.Dropout2d(p))
        
        self.feats_extractor = nn.Sequential(*layers)
        
        self.flat_feats = self.get_flat_feats(
            self.input_shape,
            self.feats_extractor
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(self.flat_feats, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, self.output_classes)
        )
    
    @staticmethod
    def get_flat_feats(input_shape, feats_model):
        dummy_input = Variable(torch.ones(1, *input_shape))
        feats = feats_model(dummy_input)
        
        return int(np.prod(feats.size()[1:]))

    def forward(self, x):
        x = self.feats_extractor(x)
        x = x.view(-1, self.flat_feats)
        
        return self.classifier(x)


In [218]:
net = BaseNet().type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(net.parameters(), lr=1e-3,
                       weight_decay=0.0001)
torch.cuda.random.manual_seed(12345)
net.apply(reset)
train(net, loss_fn, optimizer, num_epochs=30)

False 0.9 (3, 32, 32)
Starting epoch 1 / 30


t = 100, loss = 1.5450


t = 200, loss = 1.4660


t = 300, loss = 1.2622


t = 400, loss = 1.0758


...

t = 500, loss = 0.1016


t = 600, loss = 0.0124


t = 700, loss = 0.1038


Checking accuracy on validation set


Got 820 / 1000 correct (82.00)


In [167]:
torch.save(net.state_dict(), 'best.pth.tar')

In [149]:
new_model = BaseNet().type(dtype)
x = torch.load('best.pth.tar')

In [151]:
new_model.load_state_dict(x)

In [152]:
check_accuracy(new_model, loader_val)

Checking accuracy on validation set
Got 720 / 1000 correct (72.00)


In [181]:
torch.save(net, 'net')

/home/ajinkya/anaconda2/envs/231n/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type BaseNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [182]:
m = torch.load(('net'))

In [183]:
check_accuracy(m, loader_val)

Checking accuracy on validation set
Got 743 / 1000 correct (74.30)


In [197]:
print('ajinkya.%f' % 0.0)

ajinkya.0.000000
